In [3]:
from gurobipy import *
import networkx as nx
import numpy as np
import random

# Sample Example  
    List of Truck Ids
    1 2 4 5 12

    Nodes coordinates
    1 2
    3 4
    2 5
    6 6
    1 9

    Graph Matrix Time
    0 15 0 6 75
    0 0 6 7 0
    0 20 0 5 6
    5 6 7 0 0
    2 0 8 7 0

    Graph Matrix cost
    0 15 0 6 75
    0 0 6 7 0
    0 20 0 5 6
    5 6 7 0 0
    2 0 8 7 0

    Origin Node
    1 1 2 3 2

    Destination Node
    3 5 2 2 4

    Origin time
    1 1 2 5 2

    Destination time
    100 100 100 100 70

    Maximum number of vehicles in platoon
    5

    Saving factor
    1

    Transit delay factor
    
    1

In [6]:
# Defining the problem inputs based on the example

#List of trucks (could be ids or any other reference to make a difference between them)
# trucks = [1, 2, 4, 5, 12]
nbr_trucks = 5
trucks = list(range(1,nbr_trucks+1))


#Nodes coordinates (they are not used in the problem or to calculate the distance, they are just used for displaying graphs)
# nodes_coords = [[1,2], [3,4], [2,5], [6,6], [1,9]]
nbr_nodes = 7
nodes_coords = np.random.randint(-20,20,(nbr_nodes,2)).tolist()

# Defining the graph
graph = nx.DiGraph()
for i in range(len(nodes_coords)):
    graph.add_node(i+1, coords=nodes_coords[i])
# matrix_time = [[0, 15, 0, 6, 75], [0, 0, 6, 7, 0], [0 ,20 ,0 ,5 ,6], [5 ,6 ,7, 0 ,0], [2 ,0 ,8 ,7 ,0]]
# matrix_cost = [[0, 15, 0, 6, 75], [0, 0, 6, 7, 0], [0 ,20 ,0 ,5 ,6], [5 ,6 ,7, 0 ,0], [2 ,0 ,8 ,7 ,0]]
matrix_time = np.random.randint(1,20,(nbr_nodes,nbr_nodes))
matrix_cost = np.random.randint(1,50,(nbr_nodes,nbr_nodes))
np.fill_diagonal(matrix_time, 0)
np.fill_diagonal(matrix_cost, 0)
for i in range(nbr_nodes):
    set_to_zero = random.sample(list(range(nbr_nodes)), nbr_nodes//3)
    for j in set_to_zero:
        matrix_time[i,j] = 0
        matrix_cost[i,j] = 0

for i in range(len(matrix_time)):
    for j in range(len(matrix_time[i])):
        if matrix_time[i][j] != 0:
            graph.add_edge(i+1, j+1, time=matrix_time[i][j], cost=matrix_cost[i][j])
            
# origin_nodes = [1 ,1 ,2 ,3 ,2]
# destination_nodes = [3, 5, 3, 2, 4]
# origin_time = [1, 1, 2, 5, 2]
# destination_time = [100 ,100 ,100 ,100 ,70]

origin_nodes = [random.randint(1,nbr_nodes) for i in range(nbr_trucks)]
origin_time = [random.randint(1,5) for i in range(nbr_trucks)]
max_time_required = 0
destination_nodes = []
destination_time = []
for i in range(nbr_trucks):
    origin_node = origin_nodes[i]
    reachable_nodes = list(nx.descendants(graph, origin_node))
    destination_node = random.choice(list(nx.descendants(graph, 1)))
    destination_nodes.append(destination_node)
    path = nx.shortest_path(graph, 1, 2, weight="time")
    time_cost = 0
    for i in range(len(path)-1):
        time_cost += graph.get_edge_data(path[i], path[i+1])['time']
    destination_time.append(random.randint(time_cost + 7, 100))

info_trucks = dict()
for i in range(nbr_trucks):
    info_trucks[trucks[i]] = {"Origin Node": origin_nodes[i], "Destination Node":destination_nodes[i], "Origin Time": origin_time[i],"Destination Time": destination_time[i]}

#Problem parameters
Q = 5
saving_factor = 1
transit_delay_factor = 1

# B = dict()
# for truck in trucks:
#     for node in graph.nodes:
#         if node == info_trucks[truck]["Origin Node"]:
#             B[(truck,node)] = 1
#         elif node == info_trucks[truck]["Destination Node"]:
#             B[(truck,node)] = -1
#         else:
#             B[(truck,node)] = 0

M = max([info_trucks[x]['Destination Time'] for x in info_trucks]) - max([info_trucks[x]['Origin Time'] for x in info_trucks])

# Decision Variables

In [7]:
model = Model()
lst = list(zip(*list(graph.edges)))

f = dict()
for edge in graph.edges:
    f.update(model.addVars(trucks, [edge[0]], [edge[1]], vtype = GRB.BINARY))

q = dict()
for edge in graph.edges:
    q.update(model.addVars(trucks, trucks,[edge[0]], [edge[1]], vtype = GRB.BINARY))

e = dict()
for edge in graph.edges:
    e.update(model.addVars(trucks,[edge[0]], [edge[1]], vtype = GRB.CONTINUOUS))
w = model.addVars(trucks, list(graph.nodes), vtype = GRB.CONTINUOUS)
model.update()


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-08-12
Using license file C:\Users\walid\gurobi.lic


# Objective function

In [8]:
eta = 1
beta = 1
fuel_cost = quicksum(graph.get_edge_data(edge[0],edge[1])['cost']*f[truck,edge[0],edge[1]] for edge in graph.edges for truck in trucks)
platoon_benefit = quicksum(graph.get_edge_data(edge[0],edge[1])['cost']*q[truck1, truck2,edge[0],edge[1]] for edge in graph.edges for truck1 in trucks for truck2 in trucks)
delay_cost = quicksum(w)

objective_function = fuel_cost - eta*platoon_benefit + beta*delay_cost
model.setObjective(objective_function, GRB.MINIMIZE)

# Constraints

In [9]:
#Constraint 1
for truck in trucks:
    origin_node = info_trucks[truck]['Origin Node']
    destination_node = info_trucks[truck]['Destination Node']
    if origin_node != destination_node:
        children = [n for n in graph.neighbors(origin_node)]
        parents = [n for n in graph.predecessors(destination_node) if n!=origin_node]
        model.addConstr(quicksum(f[truck,origin_node,child] for child in children) == 1)
        model.addConstr(quicksum(f[truck,parent,destination_node] for parent in parents) == 1)
        for node in graph.nodes:
            if node != origin_node and node != destination_node:
                children = [n for n in graph.neighbors(node)]
                parents = [n for n in graph.predecessors(node)]
                model.addConstr(quicksum(f[truck,node,child] for child in children) == quicksum(f[truck,parent,node] for parent in parents))
for edge in graph.edges:
    if (edge[1],edge[0]) in graph.edges:
        model.addConstrs(f[truck,edge[0],edge[1]] + f[truck, edge[1], edge[0]] <= 1 for truck in trucks)
#Constraint 2
for edge in graph.edges:
    model.addConstrs(-M*(1-q[truck1,truck2, edge[0], edge[1]]) <= e[truck1, edge[0], edge[1]] - e[truck2, edge[0] ,edge[1]] for truck1 in trucks for truck2 in trucks)
    model.addConstrs(e[truck1, edge[0], edge[1]] - e[truck2, edge[0] ,edge[1]] <= M*(1-q[truck1,truck2, edge[0], edge[1]]) for truck1 in trucks for truck2 in trucks)

# #Constraint 3
for edge in graph.edges:
    for i in range(nbr_trucks-1):
        for j in range(i, nbr_trucks):
            model.addConstr(q[trucks[i],trucks[j], edge[0], edge[1]] == 0)
    model.addConstrs(quicksum(q[truck1,truck2, edge[0], edge[1]] for truck1 in trucks) <= 1 for truck2 in trucks)

#Constraint 4
for edge in graph.edges:
    model.addConstrs(quicksum(q[truck1,truck2, edge[0], edge[1]] for truck1 in trucks) <= (Q-1)*(1-quicksum(q[truck2,truck1, edge[0], edge[1]] for truck1 in trucks)) for truck2 in trucks)

#Constraint 5
for edge in graph.edges:
    model.addConstrs(2*q[truck1, truck2, edge[0], edge[1]] <= f[truck1, edge[0], edge[1]] + f[truck2, edge[0], edge[1]] for truck1 in trucks for truck2 in trucks)

#Constraint 6
for truck in trucks:
    origin_node = info_trucks[truck]['Origin Node']
    origin_time = info_trucks[truck]['Origin Time']
    children = [n for n in graph.neighbors(origin_node)]
    model.addConstrs(-M*(1-f[truck, origin_node, child]) <= e[truck, origin_node, child] - origin_time - w[truck, origin_node] for child in children)
    model.addConstrs(e[truck, origin_node, child] - origin_time - w[truck, origin_node] <= M*(1-f[truck, origin_node, child]) for child in children)

#Constraint 7
for truck in trucks:
    destination_node = info_trucks[truck]['Destination Node']
    destination_time = info_trucks[truck]['Destination Time']
    parents = [n for n in graph.predecessors(destination_node)]
    model.addConstrs(-M*(1-f[truck, parent, destination_node]) <= destination_time - e[truck, parent, destination_node] - w[truck, destination_node] - graph.get_edge_data(parent,destination_node)['time']*f[truck, parent, destination_node] for parent in parents)
    model.addConstrs(destination_time - e[truck, parent, destination_node] - w[truck, destination_node] - graph.get_edge_data(parent,destination_node)['time']*f[truck, parent, destination_node] <= M*(1-f[truck, parent, destination_node]) for parent in parents)

#Constraint 8
for truck in trucks:
    origin_node = info_trucks[truck]['Origin Node']
    destination_node = info_trucks[truck]['Destination Node']
    for node_i in graph.nodes:
        children = [n for n in graph.neighbors(node_i) if n!=origin_node and n!=destination_node]
        for node_j in children:
            second_children = [n for n in graph.neighbors(node_j)]
            for node_k in second_children:
                model.addConstr(-M*(2-f[truck, node_i, node_j]-f[truck,node_j,node_k]) <= e[truck, node_j, node_k] - e[truck, node_i, node_j] - w[truck, node_j] - graph.get_edge_data(node_i, node_j)['time']*f[truck, node_i, node_j])
                model.addConstr(e[truck, node_j, node_k] - e[truck, node_i, node_j] - w[truck, node_j] - graph.get_edge_data(node_i, node_j)['time']*f[truck, node_i, node_j] <= M*(2-f[truck, node_i, node_j]-f[truck,node_j,node_k]))

# Constraint 9
for edge in graph.edges:
    model.addConstrs(e[truck, edge[0], edge[1]] <= M*f[truck, edge[0], edge[1]] for truck in trucks)

#Constraint 10
for node in graph.nodes:
    children = [n for n in graph.neighbors(node)]
    parents = [n for n in graph.predecessors(node)]
    model.addConstrs(w[truck, node] <= M*(quicksum(f[truck, node, child] for child in children) + quicksum(f[truck, parent, node] for parent in parents)) for truck in trucks)

model.update()

In [10]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4312 rows, 1085 columns and 14517 nonzeros
Model fingerprint: 0x351e6c5c
Variable types: 185 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 2642 rows and 542 columns
Presolve time: 0.07s
Presolved: 1670 rows, 543 columns, 8604 nonzeros
Variable types: 168 continuous, 375 integer (375 binary)

Root relaxation: objective 1.880490e+02, 245 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  188.04901    0   49          -  188.04901      -     -    0s
H    0     0                     362.0000000  188.04901  48.1%     -    0s
H    0     0  